# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 8 2022 8:30AM,241362,10,MSP211959,"Methapharm, Inc.",Released
1,Jul 8 2022 8:30AM,241362,10,MSP211960,"Methapharm, Inc.",Released
2,Jul 8 2022 8:30AM,241362,10,MSP211961,"Methapharm, Inc.",Released
3,Jul 8 2022 8:30AM,241362,10,MSP211962,"Methapharm, Inc.",Released
4,Jul 8 2022 8:30AM,241362,10,MSP211963,"Methapharm, Inc.",Released
5,Jul 8 2022 8:30AM,241362,10,MSP211964,"Methapharm, Inc.",Released
6,Jul 8 2022 8:30AM,241362,10,MSP211965,"Methapharm, Inc.",Released
7,Jul 8 2022 8:30AM,241362,10,MSP211966,"Methapharm, Inc.",Released
8,Jul 8 2022 8:30AM,241362,10,MSP211967,"Methapharm, Inc.",Released
9,Jul 8 2022 8:30AM,241362,10,MSP211968,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,241358,Released,1
44,241359,Released,2
45,241360,Released,2
46,241361,Released,1
47,241362,Released,17


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241358,NaN,NaN,1.0
241359,NaN,NaN,2.0
241360,NaN,NaN,2.0
241361,NaN,NaN,1.0
241362,NaN,NaN,17.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241244,0.0,0.0,1.0
241245,0.0,0.0,1.0
241246,0.0,0.0,1.0
241248,0.0,0.0,1.0
241249,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241244,0,0,1
241245,0,0,1
241246,0,0,1
241248,0,0,1
241249,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241244,0,0,1
1,241245,0,0,1
2,241246,0,0,1
3,241248,0,0,1
4,241249,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241244,,,1
1,241245,,,1
2,241246,,,1
3,241248,,,1
4,241249,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 8 2022 8:30AM,241362,10,"Methapharm, Inc."
17,Jul 8 2022 8:30AM,241356,10,ISDIN Corporation
20,Jul 8 2022 8:30AM,241355,10,ISDIN Corporation
45,Jul 8 2022 8:30AM,241358,10,ISDIN Corporation
81,Jul 8 2022 8:30AM,241359,10,Eywa Pharma Inc.
83,Jul 8 2022 8:30AM,241360,10,Beach Products Inc
84,Jul 8 2022 8:30AM,241361,10,Emerginnova
85,Jul 8 2022 8:30AM,241360,10,Beach Patient Assistance
87,Jul 7 2022 3:25PM,241353,10,ISDIN Corporation
97,Jul 7 2022 3:03PM,241351,18,Eye Pharma Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 8 2022 8:30AM,241362,10,"Methapharm, Inc.",,,17
1,Jul 8 2022 8:30AM,241356,10,ISDIN Corporation,,,12
2,Jul 8 2022 8:30AM,241355,10,ISDIN Corporation,,,52
3,Jul 8 2022 8:30AM,241358,10,ISDIN Corporation,,,1
4,Jul 8 2022 8:30AM,241359,10,Eywa Pharma Inc.,,,2
5,Jul 8 2022 8:30AM,241360,10,Beach Products Inc,,,2
6,Jul 8 2022 8:30AM,241360,10,Beach Patient Assistance,,,2
7,Jul 8 2022 8:30AM,241361,10,Emerginnova,,,1
8,Jul 7 2022 3:25PM,241353,10,ISDIN Corporation,,,10
9,Jul 7 2022 3:03PM,241351,18,Eye Pharma Inc,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 8:30AM,241362,10,"Methapharm, Inc.",17,,
1,Jul 8 2022 8:30AM,241356,10,ISDIN Corporation,12,,
2,Jul 8 2022 8:30AM,241355,10,ISDIN Corporation,52,,
3,Jul 8 2022 8:30AM,241358,10,ISDIN Corporation,1,,
4,Jul 8 2022 8:30AM,241359,10,Eywa Pharma Inc.,2,,
5,Jul 8 2022 8:30AM,241360,10,Beach Products Inc,2,,
6,Jul 8 2022 8:30AM,241360,10,Beach Patient Assistance,2,,
7,Jul 8 2022 8:30AM,241361,10,Emerginnova,1,,
8,Jul 7 2022 3:25PM,241353,10,ISDIN Corporation,10,,
9,Jul 7 2022 3:03PM,241351,18,Eye Pharma Inc,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 8:30AM,241362,10,"Methapharm, Inc.",17,,
1,Jul 8 2022 8:30AM,241356,10,ISDIN Corporation,12,,
2,Jul 8 2022 8:30AM,241355,10,ISDIN Corporation,52,,
3,Jul 8 2022 8:30AM,241358,10,ISDIN Corporation,1,,
4,Jul 8 2022 8:30AM,241359,10,Eywa Pharma Inc.,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 8:30AM,241362,10,"Methapharm, Inc.",17.0,NaN,NaN
1,Jul 8 2022 8:30AM,241356,10,ISDIN Corporation,12.0,NaN,NaN
2,Jul 8 2022 8:30AM,241355,10,ISDIN Corporation,52.0,NaN,NaN
3,Jul 8 2022 8:30AM,241358,10,ISDIN Corporation,1.0,NaN,NaN
4,Jul 8 2022 8:30AM,241359,10,Eywa Pharma Inc.,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 8:30AM,241362,10,"Methapharm, Inc.",17.0,0.0,0.0
1,Jul 8 2022 8:30AM,241356,10,ISDIN Corporation,12.0,0.0,0.0
2,Jul 8 2022 8:30AM,241355,10,ISDIN Corporation,52.0,0.0,0.0
3,Jul 8 2022 8:30AM,241358,10,ISDIN Corporation,1.0,0.0,0.0
4,Jul 8 2022 8:30AM,241359,10,Eywa Pharma Inc.,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5550149,337.0,39.0,0.0
102,965249,4.0,0.0,0.0
16,241256,1.0,0.0,0.0
18,241351,1.0,0.0,0.0
20,965285,66.0,18.0,1.0
21,2412868,9.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5550149,337.0,39.0,0.0
1,102,965249,4.0,0.0,0.0
2,16,241256,1.0,0.0,0.0
3,18,241351,1.0,0.0,0.0
4,20,965285,66.0,18.0,1.0
5,21,2412868,9.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,337.0,39.0,0.0
1,102,4.0,0.0,0.0
2,16,1.0,0.0,0.0
3,18,1.0,0.0,0.0
4,20,66.0,18.0,1.0
5,21,9.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,337.0
1,102,Released,4.0
2,16,Released,1.0
3,18,Released,1.0
4,20,Released,66.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,18,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0
Executing,39.0,0.0,0.0,0.0,18.0,1.0
Released,337.0,4.0,1.0,1.0,66.0,9.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,18,20,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,39.0,0.0,0.0,0.0,18.0,1.0
2,Released,337.0,4.0,1.0,1.0,66.0,9.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,18,20,21
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,39.0,0.0,0.0,0.0,18.0,1.0
2,Released,337.0,4.0,1.0,1.0,66.0,9.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()